In [415]:
import pandas as pd 
import numpy as np
import re 
import matplotlib.pyplot as plt
import seaborn as sns
import math
from tqdm import tqdm 

df = pd.read_csv('./Summer22_FootballTransfers.csv')
df

,name,position,age,origin_club,league_origin_club,country_origin_club,new_club,league_new_club,country_new_club,cost,date_of_transfer
0,Amine Gouiri,Centre-Forward,22.0,OGC Nice,Ligue 1,France,Stade Rennais FC,Ligue 1,France,€42.00m,2022-09-01
1,Umar Sadiq,Centre-Forward,25.0,UD Almería,LaLiga,Spain,Real Sociedad,LaLiga,Spain,€18.00m,2022-09-01
2,Carlos Soler,Central Midfield,25.0,Valencia CF,LaLiga,Spain,Paris Saint-Germain,Ligue 1,France,€50.00m,2022-09-01
3,Manuel Akanji,Centre-Back,27.0,Borussia Dortmund,Bundesliga,Germany,Manchester City,Premier League,England,€30.00m,2022-09-01
4,Wout Faes,Centre-Back,24.0,Stade Reims,Ligue 1,France,Leicester City,Premier League,England,€10.00m,2022-09-01
...,...,...,...,...,...,...,...,...,...,...,...
33620,Nursultan Nurlanuly,Central Midfield,20.0,Baikonur Kyzylorda,Pervaya Liga,Kazakhstan,Without Club,NaN,NaN,€10Th.,2022-07-01
33621,Ovidiu Agapie,Left Winger,17.0,Ripensia Timisoara U19,Romania,Romania,Ripensia Timisoara,Liga 2,Romania,€10Th.,2022-07-01
33622,Rares Butnarasu,Right-Back,16.0,Ripensia Timisoara U19,Romania,Romania,Ripensia Timisoara,Liga 2,Romania,€10Th.,2022-07-01
33623,Marin Petkov,attack,18.0,Cherno More Varna U19,Bulgaria,Bulgaria,Cherno More Varna II,Bulgaria,Bulgaria,€10Th.,2022-07-01


In [434]:
print('we will be focusing on the 6 major leagues in european football which are English, Spanish, German, Italian, French \nand Portuguese')

we will be focusing on the 6 major leagues in european football which are English, Spanish, German, Italian, French 
and Portuguese


# Checking and cleaning the dataset

In [401]:
df.shape

(33625, 11)

In [402]:
df.isna().sum() 
print(f"these NAN values would have little incidence on the main questions we have" )

these NAN values would have little incidence on the main questions we have


In [403]:
(df == '').sum()

name                   0
position               0
age                    0
origin_club            0
league_origin_club     0
country_origin_club    0
new_club               0
league_new_club        0
country_new_club       0
cost                   0
date_of_transfer       0
dtype: int64

In [404]:
df[['origin_club', 'league_origin_club', 'new_club']].value_counts().tail(30) 

origin_club                league_origin_club        new_club             
FC Rot-Weiß Koblenz        Regionalliga Südwest      SV Eintracht Trier 05    1
                                                     FV Engers 07             1
                                                     FC Stade Nyonnais        1
                                                     FC Pratteln              1
                                                     1.FC Mühlhausen          1
                                                     1.FC Bocholt             1
FC SILON Taborsko B        Czech Republic            FK Spartak Sobeslav      1
FC SILON Taborsko U19      Czech Republic            FC SILON Taborsko        1
FC Saburtalo Academy       Georgia                   FC Merani Tbilisi        1
FC Samgurali Tskaltubo     Crystalbet Erovnuli Liga  Shukura Kobuleti         1
FC San Pedro               Cote d'Ivoire             CD Leganés B             1
FC San Giorgio             Italy             

In [405]:
pd.set_option('display.max_rows', 242)
df[df['league_origin_club'] == 'Ligue 1']


,name,position,age,origin_club,league_origin_club,country_origin_club,new_club,league_new_club,country_new_club,cost,date_of_transfer
0,Amine Gouiri,Centre-Forward,22.0,OGC Nice,Ligue 1,France,Stade Rennais FC,Ligue 1,France,€42.00m,2022-09-01
4,Wout Faes,Centre-Back,24.0,Stade Reims,Ligue 1,France,Leicester City,Premier League,England,€10.00m,2022-09-01
6,Gaëtan Laborde,Centre-Forward,28.0,Stade Rennais FC,Ligue 1,France,OGC Nice,Ligue 1,France,€18.00m,2022-09-01
11,Duje Caleta-Car,Centre-Back,25.0,Olympique Marseille,Ligue 1,France,Southampton FC,Premier League,England,€16.00m,2022-09-01
12,El Bilal Touré,Centre-Forward,20.0,Stade Reims,Ligue 1,France,UD Almería,LaLiga,Spain,€6.50m,2022-09-01
13,Christopher Wooh,Centre-Back,20.0,RC Lens,Ligue 1,France,Stade Rennais FC,Ligue 1,France,€3.00m,2022-09-01
15,Idrissa Gueye,Central Midfield,32.0,Paris Saint-Germain,Ligue 1,France,Everton FC,Premier League,England,€12.00m,2022-09-01
21,Ignatius Ganago,Centre-Forward,23.0,RC Lens,Ligue 1,France,FC Nantes,Ligue 1,France,€5.00m,2022-09-01
28,Julian Draxler,Left Winger,28.0,Paris Saint-Germain,Ligue 1,France,SL Benfica,Liga Portugal,Portugal,€18.00m,2022-09-01
33,Abdou Diallo,Centre-Back,26.0,Paris Saint-Germain,Ligue 1,France,RB Leipzig,Bundesliga,Germany,€18.00m,2022-09-01


In [406]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33625 entries, 0 to 33624
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 33625 non-null  object 
 1   position             33625 non-null  object 
 2   age                  33482 non-null  float64
 3   origin_club          33625 non-null  object 
 4   league_origin_club   32696 non-null  object 
 5   country_origin_club  32696 non-null  object 
 6   new_club             33625 non-null  object 
 7   league_new_club      28055 non-null  object 
 8   country_new_club     28055 non-null  object 
 9   cost                 33625 non-null  object 
 10  date_of_transfer     33625 non-null  object 
dtypes: float64(1), object(10)
memory usage: 2.8+ MB


In [407]:
df[df['origin_club'] == 'Paris Saint-Germain']


,name,position,age,origin_club,league_origin_club,country_origin_club,new_club,league_new_club,country_new_club,cost,date_of_transfer
15,Idrissa Gueye,Central Midfield,32.0,Paris Saint-Germain,Ligue 1,France,Everton FC,Premier League,England,€12.00m,2022-09-01
28,Julian Draxler,Left Winger,28.0,Paris Saint-Germain,Ligue 1,France,SL Benfica,Liga Portugal,Portugal,€18.00m,2022-09-01
33,Abdou Diallo,Centre-Back,26.0,Paris Saint-Germain,Ligue 1,France,RB Leipzig,Bundesliga,Germany,€18.00m,2022-09-01
180,Garissone Innocent,Goalkeeper,22.0,Paris Saint-Germain,Ligue 1,France,KAS Eupen,Jupiler Pro League,Belgium,€300Th.,2022-09-01
211,Anfane Ahamada,Defensive Midfield,20.0,Paris Saint-Germain,Ligue 1,France,FC Martigues,Championnat National,France,€100Th.,2022-09-01
286,Layvin Kurzawa,Left-Back,29.0,Paris Saint-Germain,Ligue 1,France,Fulham FC,Premier League,England,€5.00m,2022-09-01
695,Leandro Paredes,Defensive Midfield,28.0,Paris Saint-Germain,Ligue 1,France,Juventus FC,Serie A,Italy,€17.00m,2022-08-31
1027,Tidjany Touré,Left Midfield,20.0,Paris Saint-Germain,Ligue 1,France,Feyenoord U21,O21 Div. 1 Fall,Netherlands,€100Th.,2022-08-31
1032,Edouard Michut,Attacking Midfield,19.0,Paris Saint-Germain,Ligue 1,France,Sunderland AFC,Championship,England,€3.00m,2022-08-31
1745,Ander Herrera,Central Midfield,33.0,Paris Saint-Germain,Ligue 1,France,Athletic Bilbao,LaLiga,Spain,€10.00m,2022-08-27


In [82]:
#working with regex

# liste_cost = []

# for x in df['cost']:
#     liste_cost.append(x) 
    

# costs_1 = '''€12.00m 
# €6.00m 
# €5.00m 
# €500Th. 
# €200Th. 
# €250Th 
# €200Th.
# €8.00m
# €18.00m'''

    
# pattern = re.compile(r'(\d{2}[a-z]|\d{3}[A-Z][a-z].)')
# matches = pattern.finditer(costs_1)

# for match in matches:
#     print(match) 

# costs_1[67:69]

<re.Match object; span=(4, 7), match='00m'>
<re.Match object; span=(12, 15), match='00m'>
<re.Match object; span=(20, 23), match='00m'>
<re.Match object; span=(26, 32), match='500Th.'>
<re.Match object; span=(35, 41), match='200Th.'>
<re.Match object; span=(44, 50), match='250Th '>
<re.Match object; span=(52, 58), match='200Th.'>
<re.Match object; span=(62, 65), match='00m'>
<re.Match object; span=(70, 73), match='00m'>


'18'

In [93]:
# def clean_costs(cost):
#     if re.search(r'(\d{2}[a-z]|\d{3}[A-Z][a-z].)',cost):
#         pos = re.search(r'(\d{2}[a-z]|\d{3}[A-Z][a-z].)',cost).start()
#         return cost['yes']
#     else: 
#         return cost
    
# df['cost']


0        €42.00m
1        €18.00m
2        €50.00m
3        €30.00m
4        €10.00m
          ...   
33620     €10Th.
33621     €10Th.
33622     €10Th.
33623     €10Th.
33624     €25Th.
Name: cost, Length: 33625, dtype: object

In [416]:
df['cost'] = df['cost'].str.replace(r'[.]\d{2}[a-z]$', '000000', regex=True)
df['cost'] = df['cost'].str.replace('[A-Z][a-z].', '0000', regex=True)
df['cost'] = df['cost'].str.replace(r'^.', '', regex=True)
df['cost'] = df['cost'].astype(int)

In [417]:
df 

,name,position,age,origin_club,league_origin_club,country_origin_club,new_club,league_new_club,country_new_club,cost,date_of_transfer
0,Amine Gouiri,Centre-Forward,22.0,OGC Nice,Ligue 1,France,Stade Rennais FC,Ligue 1,France,42000000,2022-09-01
1,Umar Sadiq,Centre-Forward,25.0,UD Almería,LaLiga,Spain,Real Sociedad,LaLiga,Spain,18000000,2022-09-01
2,Carlos Soler,Central Midfield,25.0,Valencia CF,LaLiga,Spain,Paris Saint-Germain,Ligue 1,France,50000000,2022-09-01
3,Manuel Akanji,Centre-Back,27.0,Borussia Dortmund,Bundesliga,Germany,Manchester City,Premier League,England,30000000,2022-09-01
4,Wout Faes,Centre-Back,24.0,Stade Reims,Ligue 1,France,Leicester City,Premier League,England,10000000,2022-09-01
...,...,...,...,...,...,...,...,...,...,...,...
33620,Nursultan Nurlanuly,Central Midfield,20.0,Baikonur Kyzylorda,Pervaya Liga,Kazakhstan,Without Club,NaN,NaN,100000,2022-07-01
33621,Ovidiu Agapie,Left Winger,17.0,Ripensia Timisoara U19,Romania,Romania,Ripensia Timisoara,Liga 2,Romania,100000,2022-07-01
33622,Rares Butnarasu,Right-Back,16.0,Ripensia Timisoara U19,Romania,Romania,Ripensia Timisoara,Liga 2,Romania,100000,2022-07-01
33623,Marin Petkov,attack,18.0,Cherno More Varna U19,Bulgaria,Bulgaria,Cherno More Varna II,Bulgaria,Bulgaria,100000,2022-07-01


In [207]:
df[df['origin_club'] =='Chelsea FC']

,name,position,age,origin_club,league_origin_club,country_origin_club,new_club,league_new_club,country_new_club,cost,date_of_transfer
9,Billy Gilmour,Central Midfield,21.0,Chelsea FC,Premier League,England,Brighton & Hove Albion,Premier League,England,12000000,2022-09-01
328,Kenedy,Left Winger,26.0,Chelsea FC,Premier League,England,Real Valladolid CF,LaLiga,Spain,7000000,2022-09-01
380,Ethan Ampadu,Centre-Back,21.0,Chelsea FC,Premier League,England,Spezia Calcio,Serie A,Italy,13000000,2022-09-01
538,Marcos Alonso,Left-Back,31.0,Chelsea FC,Premier League,England,Without Club,NaN,NaN,12000000,2022-09-01
916,Abdul-Rahman Baba,Left-Back,28.0,Chelsea FC,Premier League,England,Reading FC,Championship,England,2000000,2022-08-31
1341,Callum Hudson-Odoi,Right Winger,21.0,Chelsea FC,Premier League,England,Bayer 04 Leverkusen,Bundesliga,Germany,25000000,2022-08-30
1581,Ross Barkley,Central Midfield,28.0,Chelsea FC,Premier League,England,Without Club,NaN,NaN,12000000,2022-08-29
2575,Emerson,Left-Back,28.0,Chelsea FC,Premier League,England,West Ham United,Premier League,England,14000000,2022-08-23
3986,Charly Musonda Jr.,Attacking Midfield,25.0,Chelsea FC,Premier League,England,Levante UD,LaLiga2,Spain,9000000,2022-08-16
5235,Malang Sarr,Centre-Back,23.0,Chelsea FC,Premier League,England,AS Monaco,Ligue 1,Monaco,8000000,2022-08-10


# What club purchased the most? 

In [418]:
df 
df_1 = df.groupby(['origin_club', 'league_origin_club']).agg({'cost':'sum'})
df_1.sort_values('cost', ascending=False).head(10).reset_index()

,origin_club,league_origin_club,cost
0,Chelsea FC,Premier League,302000000
1,Atalanta BC,Serie A,259100000
2,Juventus FC,Serie A,238000000
3,Manchester City,Premier League,234500000
4,Ajax Amsterdam,Eredivisie,209000000
5,Borussia Dortmund,Bundesliga,201000000
6,SSC Napoli,Serie A,193250000
7,Paris Saint-Germain,Ligue 1,192500000
8,RB Leipzig,Bundesliga,179000000
9,Sevilla FC,LaLiga,175000000


# What league earned the most?

In [419]:
df_2 = df.groupby('league_origin_club').agg({'cost':'sum'})
df_2.sort_values('cost', ascending=False).head(10)

,cost
league_origin_club,
Premier League,2439500000
Serie A,2355500000
Bundesliga,1902950000
Ligue 1,1409500000
LaLiga,1377000000
Premier Liga,1278000000
Championship,1037000000
Liga Portugal,1022750000
Eredivisie,877000000


# Which club bought the most?

In [420]:
df[['new_club', 'league_new_club', 'country_new_club']].value_counts()

new_club           league_new_club   country_new_club
Potenza Calcio     Serie C - C       Italy               26
US Cremonese       Serie A           Italy               23
Rotor Volgograd    2 League Group 1  Russia              23
RFK Novi Sad 1921  Prva liga Srbije  Serbia              23
Tuzlaspor          1.Lig             Turkey              23
                                                         ..
Rana FK            Norway            Norway               1
FC Loon-Plage      France            France               1
Rangers SC Delhi   India             India                1
Rangers de Talca   Primera B         Chile                1
Ünye 1957 Spor     Turkey            Turkey               1
Length: 6315, dtype: int64

## Which club bought the most players per league?

In [227]:
#df[(df['new_club']) & (df['league_new_club'] == 'Serie A')]
df_serie_a = df[df['league_new_club'] == 'Serie A'] 
df_serie_a['new_club'].value_counts()

US Cremonese           23
AC Monza               23
US Lecce               22
US Salernitana 1919    22
Hellas Verona          20
FC Empoli              18
UC Sampdoria           17
Udinese Calcio         14
Bologna FC 1909        12
Torino FC              12
US Sassuolo            11
Spezia Calcio          10
SSC Napoli             10
SS Lazio               10
Juventus FC            10
Inter Milan             9
AC Milan                8
AS Roma                 7
Atalanta BC             7
ACF Fiorentina          6
Name: new_club, dtype: int64

In [229]:
df_premier_league = df[(df['league_new_club'] == 'Premier League') & (df['country_new_club'] == 'England')] 
df_premier_league['new_club'].value_counts()

Nottingham Forest          21
Fulham FC                  11
Chelsea FC                 10
Southampton FC              9
West Ham United             9
Manchester United           9
Tottenham Hotspur           8
Everton FC                  8
Aston Villa                 7
Leeds United                7
Brentford FC                6
AFC Bournemouth             6
Wolverhampton Wanderers     6
Newcastle United            5
Manchester City             5
Brighton & Hove Albion      5
Arsenal FC                  5
Liverpool FC                4
Crystal Palace              4
Leicester City              2
Name: new_club, dtype: int64

In [451]:
df_ligue_1 = df[(df['league_new_club'] == 'Ligue 1') & (df['country_new_club'] == 'France')]
df_ligue_1['new_club'].value_counts() 

df_bundesliga =  df[(df['league_new_club'] == 'Bundesliga') & (df['country_new_club'] == 'Germany')]
df_bundesliga['new_club'].value_counts() 

df_la_liga= df[(df['league_new_club'] == 'LaLiga') & (df['country_new_club'] == 'Spain')]
df_la_liga['new_club'].value_counts() 

df_liga_portugal = df[(df['league_new_club'] == 'Liga Portugal') & (df['country_new_club'] == 'Portugal')]
df_liga_portugal['new_club'].value_counts() 


CD Santa Clara          18
GD Estoril Praia        18
FC Famalicão            15
Sporting CP             14
Casa Pia AC             14
SL Benfica              13
Rio Ave FC              12
CS Marítimo             12
Gil Vicente FC          12
Boavista FC             12
Vitória Guimarães SC    12
GD Chaves               11
FC Vizela               10
FC Paços de Ferreira    10
Portimonense SC         10
FC Arouca               10
SC Braga                 8
FC Porto                 7
Name: new_club, dtype: int64

In [421]:
#just type in the league name and get the clubs in order of most bought players
list_league
y = input()

df[df['league_new_club'] == y]['new_club'].value_counts()



Bundesliga


FC Schalke 04               17
Rapid Vienna                15
VfL Bochum                  12
Eintracht Frankfurt         11
Wolfsberger AC              11
SK Austria Klagenfurt       11
Austria Vienna              11
Hertha BSC                  10
1.FSV Mainz 05              10
1.FC Union Berlin           10
Borussia Dortmund           10
LASK                         9
1. FC Köln                   9
SC Austria Lustenau          9
RB Leipzig                   8
Red Bull Salzburg            8
VfB Stuttgart                8
Bayern Munich                7
FC Augsburg                  7
SK Sturm Graz                7
SV Ried                      7
SC Freiburg                  6
WSG Tirol                    6
SV Werder Bremen             6
TSV Hartberg                 6
TSG 1899 Hoffenheim          6
SCR Altach                   5
VfL Wolfsburg                5
Borussia Mönchengladbach     5
Bayer 04 Leverkusen          2
Name: new_club, dtype: int64

In [438]:
list_league

['Ligue 1',
 'LaLiga',
 'Premier League',
 'Super League 1',
 'Süper Lig',
 'Liga Portugal',
 'Serie A',
 'Premier Liga',
 'Bundesliga',
 'Premiership',
 'Serie B',
 'League One',
 "Ligat ha'Al",
 'Primera Federación - Gr. II',
 'Jupiler Pro League',
 'II. Liga',
 'Serie C - C',
 'Ligue 2',
 'Serie C - B',
 'Italy',
 'Spain',
 'LaLiga2',
 'Serie C - A',
 'Persian Gulf Pro League',
 'Championship',
 'Oberliga Westfalen',
 'Segunda Federación - Gr. IV',
 '1.Division',
 'Greece',
 'J1 League',
 'Primera Federación - Gr. I',
 'Tunisia',
 'Cymru Premier',
 'Regionalliga West',
 '1. CFL',
 'NB II.',
 'Hong Kong Premier League',
 'Segunda Federación - Gr. V',
 'Indian Super League',
 'Liga 2',
 'Regionalliga Bayern',
 'Championnat National',
 'Serie D - I',
 'League Two',
 'Ukraine',
 '3. Liga',
 '2.Lig Kirmizi',
 'Oberliga Hamburg',
 'Russia',
 '2.Lig Beyaz',
 '3.Lig Grup 1',
 'Promotion League',
 'Botola Pro Inwi',
 'Super liga Srbije',
 'Portugal',
 'Hungary',
 'Liga Sabseg',
 'Fortuna 1 L

In [344]:
import time
startTime = time.time()

#####your python script#####

executionTime = (time.time() - startTime)  
print('Execution time in seconds: ' + str(executionTime))

list_countries = list(df['country_new_club'].unique()) 

len(list_countries)

list_league = list(df['league_new_club'].unique())
len(list_league)


Execution time in seconds: 8.416175842285156e-05


464

In [361]:
df_premier_league

,name,position,age,origin_club,league_origin_club,country_origin_club,new_club,league_new_club,country_new_club,cost,date_of_transfer
3,Manuel Akanji,Centre-Back,27.0,Borussia Dortmund,Bundesliga,Germany,Manchester City,Premier League,England,30000000,2022-09-01
4,Wout Faes,Centre-Back,24.0,Stade Reims,Ligue 1,France,Leicester City,Premier League,England,10000000,2022-09-01
5,Leander Dendoncker,Defensive Midfield,27.0,Wolverhampton Wanderers,Premier League,England,Aston Villa,Premier League,England,28000000,2022-09-01
7,Pierre-Emerick Aubameyang,Centre-Forward,33.0,FC Barcelona,LaLiga,Spain,Chelsea FC,Premier League,England,15000000,2022-09-01
9,Billy Gilmour,Central Midfield,21.0,Chelsea FC,Premier League,England,Brighton & Hove Albion,Premier League,England,12000000,2022-09-01
10,James Garner,Defensive Midfield,21.0,Manchester United U21,Premier League 2,England,Everton FC,Premier League,England,7000000,2022-09-01
11,Duje Caleta-Car,Centre-Back,25.0,Olympique Marseille,Ligue 1,France,Southampton FC,Premier League,England,16000000,2022-09-01
15,Idrissa Gueye,Central Midfield,32.0,Paris Saint-Germain,Ligue 1,France,Everton FC,Premier League,England,12000000,2022-09-01
18,Carlos Vinícius,Centre-Forward,27.0,SL Benfica,Liga Portugal,Portugal,Fulham FC,Premier League,England,8000000,2022-09-01
19,Arthur Melo,Defensive Midfield,26.0,Juventus FC,Serie A,Italy,Liverpool FC,Premier League,England,18000000,2022-09-01


## Which club earned the most in the premier league through sales?


In [382]:
list_league
list_countries[0:7]

for i in range(0,7):
    #print(f"df_{list_league[i]}")= [])
    #df_(f"{list_league[i]}").append(df[df['league_origin_club'] == list_league[i]])

NameError: name 'df_' is not defined

In [422]:
list_league
y = input()

df[df['league_origin_club'] == y] 

Bundesliga


,name,position,age,origin_club,league_origin_club,country_origin_club,new_club,league_new_club,country_new_club,cost,date_of_transfer
3,Manuel Akanji,Centre-Back,27.0,Borussia Dortmund,Bundesliga,Germany,Manchester City,Premier League,England,30000000,2022-09-01
30,Aster Vranckx,Central Midfield,19.0,VfL Wolfsburg,Bundesliga,Germany,AC Milan,Serie A,Italy,10000000,2022-09-01
38,Krzysztof Piatek,Centre-Forward,27.0,Hertha BSC,Bundesliga,Germany,US Salernitana 1919,Serie A,Italy,12000000,2022-09-01
49,Ajdin Hrustic,Attacking Midfield,26.0,Eintracht Frankfurt,Bundesliga,Germany,Hellas Verona,Serie A,Italy,3000000,2022-09-01
85,John Anthony Brooks,Centre-Back,29.0,VfL Wolfsburg,Bundesliga,Germany,SL Benfica,Liga Portugal,Portugal,7000000,2022-09-01
...,...,...,...,...,...,...,...,...,...,...,...
31738,Amankwah Forson,Attacking Midfield,19.0,Red Bull Salzburg,Bundesliga,Austria,SCR Altach,Bundesliga,Austria,7000000,2022-07-01
31777,Fredy Valencia,Defensive Midfield,20.0,LASK,Bundesliga,Austria,LASK Amateure OÖ,Regional League Central,Austria,2250000,2022-07-01
31883,Daouda Guindo,Left-Back,19.0,Red Bull Salzburg,Bundesliga,Austria,FC St. Gallen 1879,Super League,Switzerland,1000000,2022-07-01
32561,Justin Che,Centre-Back,18.0,TSG 1899 Hoffenheim,Bundesliga,Germany,TSG 1899 Hoffenheim II,Regionalliga Südwest,Germany,3000000,2022-07-01


In [423]:
#df_premier_league.groupby('country_origin_club').get_group('England')
q_3_2 = df_premier_league.groupby(['origin_club', 'league_origin_club','country_origin_club']).agg({'cost':'sum'}).sort_values('cost', ascending=False).reset_index()
q_3_2[(q_3_2['country_origin_club'] == 'England') & (q_3_2['league_origin_club'] == 'Premier League')]

,origin_club,league_origin_club,country_origin_club,cost
1,Manchester City,Premier League,England,145000000
3,Brighton & Hove Albion,Premier League,England,86000000
4,Wolverhampton Wanderers,Premier League,England,70000000
5,Leeds United,Premier League,England,68000000
13,Everton FC,Premier League,England,48000000
17,Manchester United,Premier League,England,43000000
20,Leicester City,Premier League,England,40000000
28,Southampton FC,Premier League,England,31000000
32,Chelsea FC,Premier League,England,26000000
35,Aston Villa,Premier League,England,23000000


In [393]:
df_premier_league.groupby('origin_club').agg({'cost':'sum'}).sort_values('cost', ascending=False)

,cost
origin_club,
Borussia Dortmund,180000000
Manchester City,145000000
Burnley FC,87000000
Brighton & Hove Albion,86000000
Wolverhampton Wanderers,70000000
Leeds United,68000000
Atalanta BC,68000000
Ajax Amsterdam,67000000
Sporting CP,65000000


## Which club spent the most in premier league?

In [263]:
df_premier_league.groupby('new_club').agg({'cost':'sum'}).sort_values('cost', ascending=False).sum()

cost    2391250000
dtype: int64

In [309]:
for j in tqdm(range(0,464)):
    print(f"df_{list_clubs[j]}['new_club']")

100%|██████████████████████████████████████| 464/464 [00:00<00:00, 42082.71it/s]

df_Ligue 1['new_club'].value_counts()
df_LaLiga['new_club'].value_counts()
df_Premier League['new_club'].value_counts()
df_Super League 1['new_club'].value_counts()
df_Süper Lig['new_club'].value_counts()
df_Liga Portugal['new_club'].value_counts()
df_Serie A['new_club'].value_counts()
df_Premier Liga['new_club'].value_counts()
df_Bundesliga['new_club'].value_counts()
df_Premiership['new_club'].value_counts()
df_Serie B['new_club'].value_counts()
df_League One['new_club'].value_counts()
df_Ligat ha'Al['new_club'].value_counts()
df_Primera Federación - Gr. II['new_club'].value_counts()
df_Jupiler Pro League['new_club'].value_counts()
df_II. Liga['new_club'].value_counts()
df_Serie C - C['new_club'].value_counts()
df_Ligue 2['new_club'].value_counts()
df_Serie C - B['new_club'].value_counts()
df_Italy['new_club'].value_counts()
df_Spain['new_club'].value_counts()
df_LaLiga2['new_club'].value_counts()
df_Serie C - A['new_club'].value_counts()
df_Persian Gulf Pro League['new_club'].value_c

# Which player cost the most?

In [426]:
df[['name', 'position','cost', 'league_origin_club']].sort_values('cost', ascending=False).head(10)

,name,position,cost,league_origin_club
19252,Erling Haaland,Centre-Forward,150000000,Bundesliga
19261,Sadio Mané,Left Winger,70000000,Premier League
15126,Raheem Sterling,Left Winger,70000000,Premier League
19303,Romelu Lukaku,Centre-Forward,70000000,Premier League
12700,Matthijs de Ligt,Centre-Back,70000000,Serie A
19255,Federico Chiesa,Right Winger,65000000,Serie A
9425,Jules Koundé,Centre-Back,60000000,LaLiga
19250,Aurélien Tchouameni,Defensive Midfield,60000000,Ligue 1
1152,Fabián Ruiz,Central Midfield,55000000,Serie A
19251,Darwin Núñez,Centre-Forward,55000000,Liga Portugal


## Per league

In [430]:
df_serie_a[['name', 'position','cost', 'league_origin_club']].sort_values('cost', ascending=False).head(10)

,name,position,cost,league_origin_club
19303,Romelu Lukaku,Centre-Forward,70000000,Premier League
19255,Federico Chiesa,Right Winger,65000000,Serie A
16064,Paul Pogba,Central Midfield,48000000,Premier League
12316,Paulo Dybala,Second Striker,35000000,Serie A
12250,Bremer,Centre-Back,35000000,Serie A
3232,Tanguy Ndombélé,Central Midfield,30000000,Premier League
7475,Charles De Ketelaere,Attacking Midfield,30000000,Jupiler Pro League
19276,André Zambo Anguissa,Defensive Midfield,30000000,Premier League
3100,Giacomo Raspadori,Second Striker,28000000,Serie A
19269,Merih Demiral,Centre-Back,25000000,Serie A


In [431]:
df_premier_league[['name', 'position','cost', 'league_origin_club']].sort_values('cost', ascending=False).head(10)

,name,position,cost,league_origin_club
19252,Erling Haaland,Centre-Forward,150000000,Bundesliga
15126,Raheem Sterling,Left Winger,70000000,Premier League
19251,Darwin Núñez,Centre-Forward,55000000,Liga Portugal
18501,Kalvin Phillips,Defensive Midfield,50000000,Premier League
18500,Gabriel Jesus,Centre-Forward,50000000,Premier League
1151,Cristian Romero,Centre-Back,48000000,Serie A
19253,Richarlison,Centre-Forward,48000000,Premier League
5850,Gonçalo Guedes,Left Winger,40000000,LaLiga
19262,Diego Carlos,Centre-Back,40000000,LaLiga
625,Wesley Fofana,Centre-Back,40000000,Premier League


In [432]:
df_ligue_1[['name', 'position','cost', 'league_origin_club']].sort_values('cost', ascending=False).head(10)

,name,position,cost,league_origin_club
1152,Fabián Ruiz,Central Midfield,55000000,Serie A
2,Carlos Soler,Central Midfield,50000000,LaLiga
0,Amine Gouiri,Centre-Forward,42000000,Ligue 1
19256,Nuno Mendes,Left-Back,40000000,Liga Portugal
19254,Vitinha,Central Midfield,30000000,Liga Portugal
14027,Hugo Ekitiké,Centre-Forward,28000000,Ligue 1
6726,Renato Sanches,Central Midfield,28000000,Ligue 1
19287,Mattéo Guendouzi,Central Midfield,25000000,Premier League
2240,Nicolas Pépé,Right Winger,25000000,Premier League
19302,Cengiz Ünder,Right Winger,22000000,Serie A


In [448]:
df_liga_portugal[['name', 'position','cost', 'league_origin_club']].sort_values('cost', ascending=False).head(10)

,name,position,cost,league_origin_club
19301,Pedro Porro,Right-Back,25000000,Premier League
19274,David Neres,Right Winger,20000000,Premier Liga
15131,Francisco Trincão,Right Winger,20000000,LaLiga
28,Julian Draxler,Left Winger,18000000,Ligue 1
19296,Jeremiah St. Juste,Centre-Back,16000000,Bundesliga
14650,Enzo Fernández,Central Midfield,15000000,Liga Profesional
18075,David Carmo,Centre-Back,15000000,Liga Portugal
4543,Kayky,Right Winger,14000000,Premier League 2
2325,Fredrik Aursnes,Defensive Midfield,12000000,Eredivisie
11328,Gabriel Veron,Right Winger,12000000,Série A


In [449]:
df_bundesliga[['name', 'position','cost', 'league_origin_club']].sort_values('cost', ascending=False).head(10)

,name,position,cost,league_origin_club
19261,Sadio Mané,Left Winger,70000000,Premier League
12700,Matthijs de Ligt,Centre-Back,70000000,Serie A
19263,Karim Adeyemi,Right Winger,35000000,Bundesliga
5501,Timo Werner,Centre-Forward,35000000,Premier League
20736,Niklas Süle,Centre-Back,35000000,Bundesliga
19271,Ryan Gravenberch,Central Midfield,35000000,Eredivisie
17675,Sébastien Haller,Centre-Forward,35000000,Eredivisie
19270,Nico Schlotterbeck,Centre-Back,33000000,Bundesliga
6014,Angeliño,Left-Back,30000000,Bundesliga
19280,Xaver Schlager,Central Midfield,27000000,Bundesliga


In [452]:
df_la_liga[['name', 'position','cost', 'league_origin_club']].sort_values('cost', ascending=False).head(10)

,name,position,cost,league_origin_club
9425,Jules Koundé,Centre-Back,60000000,LaLiga
19250,Aurélien Tchouameni,Defensive Midfield,60000000,Ligue 1
15125,Raphinha,Right Winger,45000000,Premier League
12701,Robert Lewandowski,Centre-Forward,45000000,Bundesliga
18613,Franck Kessié,Central Midfield,45000000,Serie A
20141,Antonio Rüdiger,Centre-Back,40000000,Premier League
18573,Andreas Christensen,Centre-Back,35000000,Premier League
1325,Sergio Reguilón,Left-Back,25000000,Premier League
4398,Giovani Lo Celso,Central Midfield,22000000,Premier League
4535,Nico González,Central Midfield,20000000,LaLiga


# Which club spent the most?

In [455]:
df.describe()

,age,cost
count,33482.000000,3.362500e+04
mean,25.019951,1.731791e+06
std,4.784501,2.756059e+06
min,15.000000,8.000000e+04
25%,21.000000,5.000000e+05
50%,24.000000,1.000000e+06
75%,28.000000,2.000000e+06
max,46.000000,1.500000e+08


In [465]:
df[df['cost'] == df['cost'].max()]

,name,position,age,origin_club,league_origin_club,country_origin_club,new_club,league_new_club,country_new_club,cost,date_of_transfer
19252,Erling Haaland,Centre-Forward,21.0,Borussia Dortmund,Bundesliga,Germany,Manchester City,Premier League,England,150000000,2022-07-01


# Which position was the most sort after?

In [468]:
df['position'].value_counts()

Centre-Back           5339
Centre-Forward        5000
Central Midfield      3629
Goalkeeper            3361
Defensive Midfield    2488
Right-Back            2415
Left-Back             2361
Attacking Midfield    2303
Left Winger           2260
Right Winger          2170
midfield               532
Right Midfield         463
Left Midfield          414
attack                 316
Second Striker         314
defence                260
Name: position, dtype: int64

# Which position was the least sort after?